In [24]:
# Base class definition for RPG items
class Inventory:
    def __init__(self, name, description="", rarity="common"):
        self.name = name
        self.description = description
        self.rarity = rarity
        self._ownership = ""  # Initially, no one owns the item
    def pick_up(self, character_name):
        self._ownership = character_name
        return f"{character_name} picked up {self.name}."
    def throw_away(self):
        self._ownership = ""
        return f"{self.name} has been thrown away."
    def use(self):
        if self._ownership:
            return f"{self.name} is used."
        else:
            return f"{self.name} cannot be used as it has no owner."
    def __str__(self):
        ownership_status = self._ownership if self._ownership else "No ownership."
        return f"Item: {self.name}, Description: {self.description}, Rarity: {self.rarity}, Ownership: {ownership_status}"
    def __eq__(self, other):
        # Check if two items are the same based on name and type (or any other unique attribute)
        return isinstance(other, Inventory) and self.name == other.name and self.rarity == other.rarity


class Backpack:
    def __init__(self, owner=None):
        self.owner = owner  # The owner of the backpack
        self.items = []     # List to store items in the backpack

    def add_item(self, item):
        """Add an item to the backpack and update ownership."""
        if item not in self.items:
            item.pick_up(self.owner)  # Update ownership when item is picked up
            self.items.append(item)
            return f"{item.name} has been added to the backpack."
        return f"{item.name} is already in the backpack."

    def remove_item(self, item):
        """Remove an item from the backpack and reset ownership."""
        if item in self.items:
            self.items.remove(item)
            item.throw_away()  # Reset ownership when item is removed
            return f"{item.name} has been removed from the backpack."
        return f"{item.name} is not in the backpack."

    def view_item(self, item):
        """View a specific item in the backpack."""
        if item in self.items:
            return str(item)  # Call the __str__ method of the Inventory class
        return f"{item.name} is not in the backpack."

    def view_all_items(self):
        """View all items in the backpack."""
        if not self.items:
            return "The backpack is empty."
        return [str(item) for item in self.items]  # Return a list of item strings

    def view_items_by_type(self, item_type):
        """View items of a specific type in the backpack."""
        filtered_items = [str(item) for item in self.items if isinstance(item, item_type)]
        if not filtered_items:
            return f"There are no items of type {item_type.__name__} in the backpack."
        return filtered_items

    def __iter__(self):
        """Return an iterator for the backpack."""
        return iter(self.items)

    def __contains__(self, item):
        """Support the membership operator 'in'."""
        return item in self.items


# Subclass: Weapon
class Weapon(Inventory):
    def __init__(self, name, description, rarity, damage, weapon_type):
        super().__init__(name, description, rarity)
        self.damage = damage
        self.weapon_type = weapon_type
        self.is_equipped = False
        self.attack_mod = 1.0 if rarity in ["common", "uncommon", "epic"] else 1.15

    def __str__(self):
        base_info = super().__str__() + f", Weapon Type: {self.weapon_type}, Damage: {self.damage}, Attack Modifier: {self.attack_mod}"
        if self.rarity == "legendary":
            legendary_art = (
                "\n🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟\n"
                "        L E G E N D A R Y       \n"
                "🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟\n"
                f"{self.name}: A weapon of unimaginable power!\n"
            )
            return legendary_art + base_info
        else:
            return base_info

    def equip(self):
        self.is_equipped = True
        return f"{self.name} is equipped."

    def use(self):
        if self.is_equipped:
            attack_power = self.damage * self.attack_mod
            return f"{self.name} is used by {self._ownership}, dealing {attack_power} damage!"
        else:
            return f"{self.name} cannot be used as it is not equipped."

    def attack_move(self):
        """This will be implemented by specific weapon types"""
        raise NotImplementedError("Subclasses must implement unique attack moves")


# Subclass for Master Sword
class MasterSword(Weapon):
    def attack_move(self):
        return f"{self.name} swings with legendary might!"


# Subclass: Shield
class Shield(Inventory):
    def __init__(self, name, description, rarity, defense):
        super().__init__(name, description, rarity)
        self.defense = defense
        self.is_equipped = False
        self.is_broken = False
        self.defense_mod = 1.0 if rarity in ["common", "uncommon", "epic"] else 1.10
        self.broken_mod = 0.5

    def equip(self):
        self.is_equipped = True
        return f"{self.name} is equipped."

    def use(self):
        if self.is_broken:
            return f"{self.name} is broken and cannot be used."
        if self.is_equipped:
            total_defense = self.defense * (self.defense_mod if not self.is_broken else self.broken_mod)
            return f"{self.name} is used by {self._ownership}, blocking {total_defense} damage!"
        else:
            return f"{self.name} cannot be used as it is not equipped."

    def break_shield(self):
        if self.is_broken:
            return f"{self.name} is broken!"

    def __str__(self):
        broken_status = "Yes" if self.is_broken else "No"
        return super().__str__() + f", Defense: {self.defense}, Broken: {broken_status}"


# Subclass: Potion
class Potion(Inventory):
    def __init__(self, name, description, rarity, value, potion_type, time=0):
        super().__init__(name, description, rarity)
        self.value = value
        self.time = time
        self.potion_type = potion_type
        self.is_empty = False

    def use(self):
        if not self.is_empty:
            self.is_empty = True
            effect = f"{self.name} consumed by {self._ownership}, {self.potion_type} value: {self.value}."
            if self.potion_type == "HP":
                effect += " Health is restored."
            elif self.potion_type == "Attack":
                effect += f" Attack increased for {self.time} seconds."
            elif self.potion_type == "Defense":
                effect += f" Defense increased for {self.time} seconds."
            return effect
        else:
            return f"{self.name} is empty and cannot be used."

    def __str__(self):
        empty_status = "Yes" if self.is_empty else "No"
        return super().__str__() + f", Potion Type: {self.potion_type}, Value: {self.value}, Effective Time: {self.time}, Empty: {empty_status}"


# Function to print items in a readable format
def print_items(item_list):
    for item in item_list:
        print(item)
        print('-' * 80)  # Separator for readability

# weapon items
master_sword = Weapon(name='Master Sword', description='The legendary sword.', rarity='legendary', damage=300, weapon_type='sword')
muramasa = Weapon(name='Muramasa', description='A cursed katana with immense power.', rarity='legendary', damage=580, weapon_type='katana')
gungnir = Weapon(name='Gungnir', description='The spear of destiny.', rarity='legendary', damage=290, weapon_type='spear')
belthronding = Weapon(name='Belthronding', description='A legendary bow that deals immense damage.', rarity='legendary', damage=500, weapon_type='bow')

#other items
hp_potion = Potion(name='Healing Potion', description='Restores health by 50.', rarity='common', value=50, potion_type='HP')
broken_pot_lid = Shield(name='Wooden Lid', description='A lid made of wood, useful in cooking.', rarity='common', defense=5)
round_shield = Shield(name='Round Shield', description='A sturdy wooden shield.', rarity='uncommon', defense=10)


# backpack and master sword
beleg_backpack = Backpack(owner='Beleg')
master_sword = MasterSword(name='Master Sword', description='The legendary sword.', rarity='legendary', damage=345, weapon_type='Sword')

#added items
beleg_backpack.add_item(master_sword)
beleg_backpack.add_item(hp_potion)      
beleg_backpack.add_item(broken_pot_lid)
beleg_backpack.add_item(muramasa)      
beleg_backpack.add_item(gungnir)       
beleg_backpack.add_item(round_shield)   

# Remove an item
print(beleg_backpack.remove_item(broken_pot_lid))   # Remove broken pot lid

# Use the master sword to demonstrate its functionality
if master_sword in beleg_backpack:
    master_sword.equip()
    print(master_sword.attack_move()) 
    print(master_sword.use())          

# Iterate over items in the backpack
for item in beleg_backpack:
    if isinstance(item, Weapon):
        print(beleg_backpack.view_item(item))

Wooden Lid has been removed from the backpack.
Master Sword swings with legendary might!
Master Sword is used by Beleg, dealing 396.74999999999994 damage!

🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟
        L E G E N D A R Y       
🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟
Master Sword: A weapon of unimaginable power!
Item: Master Sword, Description: The legendary sword., Rarity: legendary, Ownership: Beleg, Weapon Type: Sword, Damage: 345, Attack Modifier: 1.15

🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟
        L E G E N D A R Y       
🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟
Muramasa: A weapon of unimaginable power!
Item: Muramasa, Description: A cursed katana with immense power., Rarity: legendary, Ownership: Beleg, Weapon Type: katana, Damage: 580, Attack Modifier: 1.15

🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟
        L E G E N D A R Y       
🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟🌟
Gungnir: A weapon of unimaginable power!
Item: Gungnir, Description: The spear of destiny., Rarity: legendary, Ownership: Beleg, Weapon Type: spear, Damage: 290, Attack Modifier: 1.15
